In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#baseline ver
VER = 6

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
#import cudf, 
import itertools
#print('We will use RAPIDS version',cudf.__version__)

# test data(valid data)

In [3]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/cris_baseline/'

In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(INPUT_DIR+'cris_baseline/otto-validation/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

#test_df
test_df = load_test()
print('Test data has shape',test_df.shape)
test_df.head()

Test data has shape (7683577, 4)


,session,aid,ts,type
0,11188591,1485015,1661165822,0
1,11188591,1562739,1661165866,0
2,11188591,1485015,1661165882,0
3,11188591,1441634,1661165891,0
4,11188591,1485015,1661165901,0


In [5]:
#test_labels
test_labels = pd.read_parquet('/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/otto-validation/test_labels.parquet')
test_labels

,session,type,ground_truth
0,11098528,clicks,[1679529]
1,11098528,carts,[1199737]
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
3,11098529,clicks,[1105029]
4,11098530,orders,[409236]
...,...,...,...
2212687,12899774,clicks,[1399483]
2212688,12899775,clicks,[1760714]
2212689,12899776,clicks,[1737908]
2212690,12899777,clicks,[384045]


# clicks candidate

## baseline

In [6]:
'''
%%time
# LOAD THREE CO-VISITATION MATRICES
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

#covisi matrix
DISK_PIECES = 4
top_20_clicks = pqt_to_dict( pd.read_parquet(INPUT_DIR+f'cris_baseline/output/top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(INPUT_DIR+f'cris_baseline/output/top_20_clicks_v{VER}_{k}.pqt') ) )

# TOP CLICKS AND ORDERS IN TEST
top_clicks = test_df.loc[test_df['type']==0,'aid'].value_counts().index.values[:20]
#top_clicks = test_df.loc[test_df['type']=='clicks','aid'].value_counts().index.values[:20]

type_weight_multipliers = {0: 1, 1: 6, 2: 3}

def suggest_clicks(df):
    # USE USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=20:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        sorted_aids = [k for k,v in aids_temp.most_common(20)]
        return sorted_aids
    # USE "CLICKS" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(20) if aid2 not in unique_aids]    
    result = unique_aids + top_aids2[:20 - len(unique_aids)]
    # USE TOP20 TEST CLICKS
    return result + list(top_clicks)[:20-len(result)]

pred_df_clicks = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(lambda x: suggest_clicks(x))
pred_df_clicks = pd.DataFrame(pred_df_clicks, columns=["labels"]).reset_index()
pred_df_clicks["labels"] = pred_df_clicks.labels.apply(lambda x: " ".join(map(str,x)))
pred_df_clicks
'''

'\n%%time\n# LOAD THREE CO-VISITATION MATRICES\ndef pqt_to_dict(df):\n    return df.groupby(\'aid_x\').aid_y.apply(list).to_dict()\n\n#covisi matrix\nDISK_PIECES = 4\ntop_20_clicks = pqt_to_dict( pd.read_parquet(INPUT_DIR+f\'cris_baseline/output/top_20_clicks_v{VER}_0.pqt\') )\nfor k in range(1,DISK_PIECES): \n    top_20_clicks.update( pqt_to_dict( pd.read_parquet(INPUT_DIR+f\'cris_baseline/output/top_20_clicks_v{VER}_{k}.pqt\') ) )\n\n# TOP CLICKS AND ORDERS IN TEST\ntop_clicks = test_df.loc[test_df[\'type\']==0,\'aid\'].value_counts().index.values[:20]\n#top_clicks = test_df.loc[test_df[\'type\']==\'clicks\',\'aid\'].value_counts().index.values[:20]\n\ntype_weight_multipliers = {0: 1, 1: 6, 2: 3}\n\ndef suggest_clicks(df):\n    # USE USER HISTORY AIDS AND TYPES\n    aids=df.aid.tolist()\n    types = df.type.tolist()\n    unique_aids = list(dict.fromkeys(aids[::-1] ))\n    # RERANK CANDIDATES USING WEIGHTS\n    if len(unique_aids)>=20:\n        weights=np.logspace(0.1,1,len(aids),base

## covisi matrix

In [17]:
%%time

# LOAD THREE CO-VISITATION MATRICES
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

#読み込み
DISK_PIECES = 4
top_20_clicks = pqt_to_dict( pd.read_parquet(INPUT_DIR+f'cris_baseline/output/top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(INPUT_DIR+f'cris_baseline/output/top_20_clicks_v{VER}_{k}.pqt') ) )

CPU times: user 34.3 s, sys: 2.97 s, total: 37.2 s
Wall time: 37.5 s


In [18]:
%%time
def covis_matrix_click_cart_order_to_clicks_timeweighting(df):
    aids = df.aid.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    results =itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks])
    results = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    return results

pred_df_clicks = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(lambda x: covis_matrix_click_cart_order_to_clicks_timeweighting(x))
pred_df_clicks = pd.DataFrame(pred_df_clicks, columns=["labels"]).reset_index()
pred_df_clicks["labels"] = pred_df_clicks.labels.apply(lambda x: " ".join(map(str,x)))
pred_df_clicks

#pred_df_clicks['labels']が空のレコードはrecall_evaluateの[int(i) for i in x.split(' ')]　でintにできなくてエラーになるので適当なリストで埋めておく。
for i in range(len(pred_df_clicks)):
  if len(pred_df_clicks['labels'][i]) == 0:
    pred_df_clicks['labels'][i] = '1 1' #２個以上の数字で空白を作らないとrecall_evaluateのx.split(' ')できなくてエラーになる。

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


CPU times: user 4min 17s, sys: 2.88 s, total: 4min 20s
Wall time: 4min 19s


## test_unique_aids

In [15]:
'''
%%time
def test_unique_aids(df):
    aids = df.aid.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    return unique_aids

pred_df_clicks = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(lambda x: test_unique_aids(x))
pred_df_clicks = pd.DataFrame(pred_df_clicks, columns=["labels"]).reset_index()
pred_df_clicks["labels"] = pred_df_clicks.labels.apply(lambda x: " ".join(map(str,x)))
pred_df_clicks
'''

CPU times: user 1min 34s, sys: 1.71 s, total: 1min 36s
Wall time: 1min 36s


,session,labels
0,11098528,11830
1,11098529,1105029
2,11098530,409236 264500
3,11098531,396199 1271998 452188 1728212 1365569 624163 1...
4,11098532,876469 7651
...,...,...
1801246,12899774,33035
1801247,12899775,1743151
1801248,12899776,548599
1801249,12899777,384045


# test top 20clicks

In [13]:
'''
%%time
top_clicks = test_df.loc[test_df['type']==0,'aid'].value_counts().index.values[:20]
top_clicks

#test top 20clicksは全てのセッションで同じ商品コード予測するだけなので、groupby周りの処理は不要だけど一応baselineのcodeに合わせる。
pred_df_clicks = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(lambda x: top_clicks)
pred_df_clicks = pd.DataFrame(pred_df_clicks, columns=["labels"]).reset_index()
pred_df_clicks["labels"] = pred_df_clicks.labels.apply(lambda x: " ".join(map(str,x)))
'''

CPU times: user 33.9 s, sys: 636 ms, total: 34.6 s
Wall time: 34.5 s


"\npred_df_clicks = test_df.copy()\npred_df_clicks['labels'] = 0\npred_df_clicks['labels'] = pred_df_clicks['labels'].apply(lambda x:list(top_clicks))\npred_df_clicks\n"

# recall

In [26]:
%%time

#clicks,cart,orderのracall計算
def recall_evaluate(test_labels, pred_df, session_types):
    sub = pred_df.copy()
    #print(sub)
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')]) #recall@20ではなくrecallなので[:20]外す
    test_labels = test_labels.loc[test_labels['type']==session_types]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1) 
    test_labels['gt_count'] = test_labels.ground_truth.str.len() #recall@20ではなくrecallなので.clip(0,20)外す
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    print(f'{session_types} recall =',recall)

#clicksの場合
#test labelはCV変わるとtest labelも変わるので引数にする。
recall_evaluate(test_labels, pred_df_clicks, 'clicks')

clicks recall = 0.3756537896731137
CPU times: user 1min 20s, sys: 2.33 s, total: 1min 23s
Wall time: 1min 22s


In [29]:
max(test_labels.ground_truth.str.len().clip(0,20))

20

In [21]:
test_labels

,session,type,ground_truth
0,11098528,clicks,[1679529]
1,11098528,carts,[1199737]
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
3,11098529,clicks,[1105029]
4,11098530,orders,[409236]
...,...,...,...
2212687,12899774,clicks,[1399483]
2212688,12899775,clicks,[1760714]
2212689,12899776,clicks,[1737908]
2212690,12899777,clicks,[384045]


In [22]:
pred_df_clicks

,session,labels
0,11098528,588923 1732105 571762 884502 1157882 876129 11...
1,11098529,459126 1339838 1544564 217742 1694360 1383767 ...
2,11098530,1603001 963957 254154 583026 167895 364155 752...
3,11098531,452188 1365569 698990 1136142 1553691 653835 6...
4,11098532,1202618 1159379 77906 1704066 1212859 669555 7...
...,...,...
1801246,12899774,1539309 819288 95488 771913 270852 743977 3149...
1801247,12899775,1760714 1255910 1163166 832192 29735 1498443 1...
1801248,12899776,1401030 1440959 1607333 1144446 364185 1150130...
1801249,12899777,1308634 1688215 703474 395762 1486067 211171 9...
